# Exercise Group - Chapter 1 and 2

Below you find exercises for end of Chapter 1 and the start of Chapter 2. These exercises will prepare you in particular for the obligatory mid-term tests. The purpose of the prepared code snippets below is to give you some helpful structure for coding by predefining some useful variable names, but you do not need to use these prepared code snippets and prepared variable names if you do not want to. Do not hestitate to use the function `help()` to print the documentation of the functions that you need. If you are stuck, please ask one of the supervisors present.

In the shared folder you will find a subfolder `data` containing important time series. You may copy them to your "*my_materials*" folder in order to import the data from there if you like.

<br><br>

# Topic 1: Backtesting the Value-at-Risk for the DAX

## Step 1: Visual Backtesting

To complete this exercise, start by running the code provided above. Next, we aim to verify whether the calculated VaR is performing as expected.

**Task 1.1**: Calculate the actual losses of the dax time series.

**Task 1.2**: Plot the different levels of VaR against the actual losses of the dax timeseries.

<details>
  <summary>Interpretation:</summary>
    
As expected, the Value-at-Risk measures become larger and larger as you increase $\alpha$ since the VaR gives an upper bound for the losses that is not surpassed with a probability of $\alpha$. Note how the Value-at-Risk adapts to shocks in volatility with a certain delay in time: You can observe this very clearly after the outbreak of the COVID-19 pandemic after around 5000 time points. Compared with the output of Week 1, Task 3.4, it is also visible that the VaR risk measure here adapts much faster and better to changes in volatility over time. This is because we dropped the assumption of iid log-returns from Week 1 and estimated $\mu$ and $\sigma$ from moving windows of data, always looking only at the data of the past year instead of at the whole history of data.

</details>

<img src="./Images/Task2_1_2.png" width="450">

Now, examine your plot and consider the following questions. Once you got your own answer, you may reveal the answers below.

(1) What do you notice? Do the Value-at-Risk measures perform well? Is there a specific level $\alpha$ that outperforms the others?
<details>
  <summary>Answer:</summary>

Compared to e.g. the risk measure from Week 1, Task 3.4, the Value-at-Risk with mean and standard deviation computed only from the past trading year adapts much better to periods of high and low volatility. Nevertheless, the Value-at-Risk risk measure always "lags behind" a certain amount of time in adapting to high stress periods in markets. On a quantitative level, the plot alone doesn’t allow us to determine whether the VaR measure is performing well.
</details><br>

(2) Why might it be more appropriate to base the calculation above on the most recent 252 trading days rather than on the entire time series?

<details>
<summary>Answer:</summary>
Using the entire dataset may overlook potential structural changes in the underlying time series. It imposes the assumption of stationarity on the distribution of the log-returns which clearly does not hold in reality since it exhibits periods of higher and of lower volatility.
</details>


## Step 2: Mathematical Backtesting

Step 1 highlights why a more rigorous mathematical approach to backtesting is essential; from the plot alone, it’s unclear if the computed VaR is effective.

Lemma 1.10 provides the foundation for this mathematical approach to backtesting the VaR. How could the result of Lemma 1.10 be used for backtesting?

<details>
  <summary>Answer:</summary>

  Lemma 1.10 indicates that VaR exceedences should be independent and quantifies the expected number of exceedences: for a 95% VaR, we expect 100%−95%=5% exceedances. If we observe significantly more or fewer exceedences, this suggests our model or method may need adjustment. But what does “more” or “fewer” mean in this context? How much deviation from the expected exceedences is acceptable?
        <details>
              <summary>Answer:</summary>
              The relative frequency of exceedences should lie within a confidence interval. A 95% confidence interval for the exceedence frequency of the 99%-VaR is for example an interval around the number $1 - 0.99 = 0.01$ of which we can be certain with a 95% probability that the losses of the underlying time series exceed the 99%-VaR with a frequency that lies within the interval. If the actually observed number of exceedences in the past happens to lie outside of the interval, this indicate a problem with the model or similar issues.
          </details>
</details>

<br>

**Task 1.3**: How many exceedences of the VaR risk measures plotted in Task 1.2 do you expect, and how many do you actually observe? Interpret your results.

$\alpha = 0.9$: &nbsp;&nbsp;&nbsp;&nbsp; 9.61% actual exceedences, 10.00% expected <br>
$\alpha = 0.95$: &nbsp;&nbsp; 6.16% actual exceedences, 5.00% expected<br>
$\alpha = 0.975$: &nbsp;3.86% actual exceedences, 2.50% expected<br>
$\alpha = 0.99$: &nbsp;&nbsp; 2.15% actual exceedences, 1.00% expected

<details>
  <summary>Interpretation:</summary>
    
Compared to the expected fraction of exceedences, our Value-at-Risk is not strict enough except in the case $\alpha = 0.9$, that is, it does not predict as many exceedences as there actually occur. This becomes more pronounced the smaller you choose $\alpha$: For $\alpha =0.99$, there are already more than twice as many exceedences as one would expect from Lemma 1.10 and a sensible 99%-VaR estimate. From a viewpoint of risk management, formula (1.5) in the lecture notes does not provide enough protection against large losses of the DAX index as desired. The reason for that is that the assumption of independent log-normally distributed log-returns does not hold in practice: First, the log-normal distribution has tails which are not heavy enough to describe the unconditional returns distribution of the DAX, and second, the import effect of volatility clusters in the time series is not incorporated into our model due to the independence assumption.

</details>

## Step 3: Design a Statistical Test

Assume that we have a sample $ν = (ν_1, ..., ν_m) \in \{0, 1\}^m$ of i.i.d. random variables with $P(ν_1 = 1) = p \in [0, 1]$ where $p$ is unknown. Hence, their sum $\sum^m_{k=1} v_k$ follows a $\mathrm{Bin}(m, p)$-distribution, i.e. a binomial distribution with $m$ experiments and success probability $p$.

A statistical test is a method used to determine whether there is enough evidence in a sample to support a specific hypothesis about a population. It involves comparing observed data against a null hypothesis (typically suggesting no effect or difference) and calculating a test statistic that reflects the difference between the observed data and expectations under the null hypothesis. We would like to test whether $p=p_0$ at some confidence level $\beta$, i.e. we want to test whether the sample $v_1, \dots, v_m$ contains enough evidence to support the hypothesis that the unknown true parameter $p$ is equal to some given $p_0$ with a confidence of 95%.

Why does a two-sided test make more sense in the risk management context of backtesting the VaR than a one-sided test?

*Hint*: In a statistical test, "two-sided" means that the test checks for significant deviations in both directions from the null hypothesis.

<details>
    <summary>Answer:</summary>
    It’s important to check whether the VaR is too lenient, resulting in more violations than expected, or too strict, which would lead to overestimating the risk and indicate a poor model fit. The first problem would result in too few reserved capital, the second problem would result in too much reserved capital from the point of view of a risk management department.
</details>

<br>

**Task 1.4**: Write a function `boundaries(beta, m, p_0)` that returns the lower and upper boundary of a two-sided confidence interval at level $\beta$ for the $\mathrm{Bin}(m, p_0)$ distribution, i.e. an interval $[\ell_l, \ell_u]$ that contains a $\mathrm{Bin}(m, p_0)$-distributed random variable with a probability of $\beta$.

**Task 1.5**: Design a two sided statistical test at significance level $\beta \in (0, 1)$ for the null hypothesis $H_0 : p = p_0$ and implement this test. 
This test is supposed to return the value 1 if the null hypothesis is rejected, and 0 otherwise.

**Task 1.6**: Apply this test at the level $\beta=0.99$ to the different levels of VaR of the DAX data. What do you observe? How large do you need to choose $\beta$ such that none of the hypotheses is rejected?

*Results for $\beta = 0.99$:* <br>
$\alpha = 0.9$: &nbsp;&nbsp;&nbsp;&nbsp; $p = 0.1$ is not rejected <br>
$\alpha = 0.95$: &nbsp;&nbsp; $p = 0.05$ is rejected<br>
$\alpha = 0.975$: &nbsp;$p = 0.025$ is rejected<br>
$\alpha = 0.99$: &nbsp;&nbsp; $p = 0.01$ is rejected <br>
<br>
One has to choose $\beta \approx 1 - 10^{-15}$ so that none of the hypotheses is rejected. In other words, the $p$-value of the last hypothesis test for $p_0 = 0.01$ is approximately as low as $10^{-15}$.

<details>
  <summary>Interpretation:</summary>
    
This very harsh rejection of the hypotheses in any of the cases $\alpha = 0.95$, $\alpha = 0.975$ and $\alpha = 0.99$ goes hand in hand with the interpretation under Task 1.3. For example, the tail of the uppermost percentile of the log-returns distribution is so different from the corresponding tail of a log-normal distribution, that the corresponding null hypothesis for $\alpha = 0.99$ is rejected at any reasonable confidence level.

</details>

<br><br>

# Topic 2: Expected Shortfall as a Risk Measure

**Task 2.1**: Write a function `ES(s, alpha, mu, sigma)` that computes the Expected Shortfall at level $\alpha$ for a loss of the form $L_{n+1} = s(1 - e^X)$ with $X \sim N(\mu, \sigma^2)$ as in Example 1.15 using equation (1.10) from the lecture notes.

**Task 2.2**: Apply this function at the level $\alpha=0.95$ to the DAX time series like for the Value-at-Risk in the provided code snippet above Topic 1, i.e. the estimation of $\mu$ and $\sigma$ should be based on the last 252 trading days. 

**Task 2.3**: Plot the actual losses, the $VaR_{95\%}$ and the $ES_{95\%}$ in a common plot. What do you observe?

<details>
  <summary>Interpretation:</summary>
    
As expected, the Expected Shortfall measure lies above the Value-at-Risk measure plotted in Task 1.2. The 95%-ES contains the average loss that you incur once the loss exceeds the 95%-VaR. The fact that very many of the exceedences of the VaR even exceed the ES speeks for the fact that the log-return distribution is very heavy-skewed in the right-tail and has comparably very much mass in percentiles way beyond the 95th percentile.

</details>

<img src="./Images/Task2_2_3.png" width="450">

Do you know a similar result as Lemma 1.10 for the Expected Shortfall? How would you test it?

<details>
  <summary>Answer:</summary>
    
There does not really exist a satisfying analogue of Lemma 1.10 for the Expected Shortfall. This is one of the largest drawbacks of the Expected Shortfall compared to the Value-at-Risk, which can be backtested completely nonparametrically as in Topic 1. To backtest a parametric estimate of the Expected Shortfall based on the log-normal distribution as above, one would first need to backtest the assumption of log-normality in the tails of the (conditional) returns distribution. The topic of correctly fitting a certain parametric distribution to the tails of the log-returns is touched upon in Chapter 3 of the lecture.

</details>

<br><br>

# Topic 3: Variance-Covariance Method

We now want to calculate the VaR for the portfolio from Week 1, Topic 2 using the Variance-covariance method. 

Consider a stock portfolio $(V_n)_{n \in \{0, \dots, N\}}$ with [4, 8, 15, 16, 23] shares of the German stocks [BMW, SAP, Volkswagen, Continental, Siemens] and suppose that you use logarithmic prices $Z_{n, i} = \log S_{n, i}$ as risk factors. For simplicity, we suppose that the risk factor changes $X_{n+1, i} = \log\big( \frac{S_{n + 1, i}}{S_{n, i}}\big)$ (i.e. the logarithmic returns of the stocks) are iid normally distributed. In this case, the linearised loss has the form $L_{n+1}^{\Delta} = -\sum_{i=1}^5 \alpha_i S_{n, i} X_{n+1, i}$ (see Section 1.2.3).

<br><br>

## Step 1: Set up the Variance-Covariance Method

**Task 3.1**: Define an array `w` and an array `c` that contains the constants $w_n \in \mathbb{R}^5$ and $c_n \in \mathbb{R}$ for each $n \geq 252$. Accordingly, `w` should have shape `(N - 252, 5)` and `c` should have shape `(N - 252, )`.

**Task 3.2**: For each $n \geq 252$, compute the mean and covariance estimates $\widehat \mu \in \mathbb{R}^5$ and $\widehat \Sigma \in \mathbb{R}^{5 \times 5}$ as in Section 2.2. However, each estimate should only be computed using the previous 252 time steps instead of the whole history of past data, as in the lecture notes. Store your estimates in an array `mu_hat` of shape `(N - 252, 5)` and in an array `Sig_hat` of shape `(N - 252, 5, 5)`. 

<br><br>

## Step 2: Compute VaR and ES Estimates

**Task 3.3**: For each $n \geq 252$, compute the VaR estimate $\widehat{\mathrm{VaR}_{0.99}}(L_{n+1}^{\Delta})$ using equation (2.2) in the lecture notes. Store your VaR estimates in an array `VaR`. Accordingly, it should have length `N - 252`.

**Task 3.4**: For each $n \geq 252$, compute the ES estimate $\widehat{\mathrm{ES}_{0.99}}(L_{n+1}^{\Delta})$ using equation (2.3) in the lecture notes. Store your ES estimates in an array `ES`. Accordingly, it should have length `N - 252`.

<br><br>

## Step 3: Visualise and Interpret your Results

**Task 3.5**: Compute the portfolio losses $L_{n+1} = -(V_{n+1} - V_n)$ for $n \geq 252$, store them in an array `losses` and plot them using the function `plt.plot`. Additionally, plot your VaR and ES estimates from the arrays `VaR` and `ES` in the same plot.

<details>
  <summary>Interpretation:</summary>
    
The interpretation of this plot is rather similar to the interpretation of the output in Task 2.3. The formulas for the variance-covariance method in Section 2.2 make the method especially well-suited when working with multidimensional risk factors whose dependence structure is well-approximated by a Gaussian copula (see Chapter 4). If risk factors are only one-dimensional, the use of the variance-covariance method is nonsensical and one should rather use equation (1.5) in the lecture notes, which then leads to the output in Task 2.3.

</details>

<img src="./Images/Task2_3_5.png" width="450">

**Task 3.6**: Compute the fraction of time points at which the actually incurred loss stored in the array `losses` exceeds the VaR estimate stored in the array `VaR`. Interpret your result.

Fraction of loss exceedences of the VaR estimate: 1.734%

**Task 3.7**: Compute the linearised losses $L^\Delta_{n+1} = -\sum_{i=1}^5 \alpha_i S_{n, i} X_{n + 1, i}$ for $n \geq 252$ and store them in an array `lin_losses`. Compute the fraction of time points at which the linearised loss stored in the array `lin_losses` exceeds the VaR estimate stored in the array `VaR`. Interpret your result.

Fraction of linearised loss exceedences of the VaR estimate: 1.816%

<details>
  <summary>Interpretation:</summary>
    
As in Task 1.3, the VaR estimate provided by the Variance-Covariance method is not strict enough to protect against 99% of the incurred losses. The actual fraction of loss exceedences is almost twice as high as one would expect, emphasising once more the non-applicability of the normal and log-normal distribution in modelling financial stock returns. Interestingly, the VaR estimate performs even worse on the linearised losses than on the actual losses, even though the Variance-Covariance method inherently works in its construction with the concept of linearised losses.

</details>